In [3]:
'''
Pretrained models

How to change the weights with TL?
'''

from keras_segmentation.pretrained import pspnet_50_ADE_20K , pspnet_101_cityscapes, pspnet_101_voc12

model = pspnet_50_ADE_20K() # load the pretrained model trained on ADE20k dataset

model = pspnet_101_cityscapes() # load the pretrained model trained on Cityscapes dataset

model = pspnet_101_voc12() # load the pretrained model trained on Pascal VOC 2012 dataset

# load any of the 3 pretrained models

out = model.predict_segmentation(
    inp="input_image.jpg",
    out_fname="out.png"
)

264151040/264149344 [==============================] - 43s 0us/step


AttributeError: 'NoneType' object has no attribute 'shape'

In [11]:
from keras_segmentation.models.unet import vgg_unet

model = vgg_unet(n_classes=51 ,  input_height=416, input_width=608  )

In [14]:
model.train(
    train_images =  "dataset1/images_prepped_train/",
    train_annotations = "dataset1/annotations_prepped_train/"#,
    #checkpoints_path = "/checkpoints/vgg_unet_1" , epochs=5 #created folder checkpoint
)

out = model.predict_segmentation(
    inp="dataset1/images_prepped_test/0016E5_07965.png",
    out_fname="/tmp/out.png"
)

import matplotlib.pyplot as plt
plt.imshow(out)

# evaluating the model 
print(model.evaluate_segmentation( inp_images_dir="dataset1/images_prepped_test/"  , annotations_dir="dataset1/annotations_prepped_test/" ) )

Verifying training dataset


100%|████████████████████████████████████████| 367/367 [00:06<00:00, 58.97it/s]

Dataset verified! 
Instructions for updating:
Please use Model.fit, which supports generators.


Epoch 1/5
 80/512 [===>..........................] - ETA: 1:16:37 - loss: 4.0889 - accuracy: 0.0573

KeyboardInterrupt: 

In [3]:
import os

In [4]:
import os
import tensorflow as tf
import cv2, os
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
#plt.rcParams['figure.figsize'] = (15, 15)

#import ochumanApi.vis as vistool
#from ochumanApi.ochuman import Poly2Mask

In [5]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation, Lambda, GlobalAveragePooling2D, concatenate
from tensorflow.keras.layers import UpSampling2D, Conv2D, Dropout, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

# from tensorflow.keras.applications.vgg16 import VGG16
# from keras.applications.vgg16 import VGG16
# from tensorflow.keras.applications.inception_v3 import InceptionV3

import numpy as np
import matplotlib.pyplot as plt
import cv2
import pickle
import random
from sklearn.model_selection import train_test_split

In [6]:
IMG_HEIGHT = 360
IMG_WIDTH = 480
epochs = 50
batch_size = 16
ImgDir = "C:\\Users\\pablo\\Google Drive\\2 TUM\\Internships\\Tutorials\\dataset1\\"

In [7]:

features_train = os.listdir(f"{ImgDir}images_prepped_train\\")
labels_train = os.listdir(f"{ImgDir}annotations_prepped_train\\")

print(len(features_train), len(labels_train))

features_test = os.listdir(f"{ImgDir}images_prepped_test\\")
labels_test = os.listdir(f"{ImgDir}annotations_prepped_test\\")

print(len(features_test), len(labels_test))

367 367
101 101


In [8]:
features_train

['0001TP_006690.png',
 '0001TP_006720.png',
 '0001TP_006750.png',
 '0001TP_006780.png',
 '0001TP_006810.png',
 '0001TP_006840.png',
 '0001TP_006870.png',
 '0001TP_006900.png',
 '0001TP_006930.png',
 '0001TP_006960.png',
 '0001TP_006990.png',
 '0001TP_007020.png',
 '0001TP_007050.png',
 '0001TP_007080.png',
 '0001TP_007110.png',
 '0001TP_007140.png',
 '0001TP_007170.png',
 '0001TP_007200.png',
 '0001TP_007230.png',
 '0001TP_007260.png',
 '0001TP_007290.png',
 '0001TP_007320.png',
 '0001TP_007350.png',
 '0001TP_007380.png',
 '0001TP_007410.png',
 '0001TP_007440.png',
 '0001TP_007470.png',
 '0001TP_007500.png',
 '0001TP_007530.png',
 '0001TP_007560.png',
 '0001TP_007590.png',
 '0001TP_007620.png',
 '0001TP_007650.png',
 '0001TP_007680.png',
 '0001TP_007710.png',
 '0001TP_007740.png',
 '0001TP_007770.png',
 '0001TP_007800.png',
 '0001TP_007830.png',
 '0001TP_007860.png',
 '0001TP_007890.png',
 '0001TP_007920.png',
 '0001TP_007950.png',
 '0001TP_007980.png',
 '0001TP_008010.png',
 '0001TP_0

In [9]:
'''
Normalization
'''
x_train = []
x_test = []
y_train = []
y_test = []


for i in range(len(features_train)):
    
    img_params = cv2.imread(os.path.join(f"{ImgDir}images_prepped_train\\",features_train[i]))
    img_params = img_params / 255.
    x_train.append(img_params)
    img_label = cv2.imread(os.path.join(f"{ImgDir}annotations_prepped_train\\",labels_train[i]))
    img_label = img_label / 255.
    y_train.append(img_label)
    
    
for i in range(len(features_test)):
    
    img_params = cv2.imread(os.path.join(f"{ImgDir}images_prepped_test\\",features_test[i]))
    img_params = img_params / 255.
    x_test.append(img_params)
    img_label = cv2.imread(os.path.join(f"{ImgDir}annotations_prepped_test\\",labels_test[i]))
    img_label = img_label / 255.
    y_test.append(img_label)
    
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [10]:
def get_model():
    in1 = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3 ))

    conv1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(in1)
    conv1 = Dropout(0.2)(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool1)
    conv2 = Dropout(0.2)(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool2)
    conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)

    conv4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool3)
    conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv4)

    up1 = concatenate([UpSampling2D((2, 2))(conv4), conv3], axis=-1)
    conv5 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up1)
    conv5 = Dropout(0.2)(conv5)
    conv5 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv5)
    
    up2 = concatenate([UpSampling2D((2, 2))(conv5), conv2], axis=-1)
    conv6 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up2)
    conv6 = Dropout(0.2)(conv6)
    conv6 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv6)

    up2 = concatenate([UpSampling2D((2, 2))(conv6), conv1], axis=-1)
    conv7 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up2)
    conv7 = Dropout(0.2)(conv7)
    conv7 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv7)
    segmentation = Conv2D(3, (1, 1), activation='sigmoid', name='seg')(conv7)

    model = Model(inputs=[in1], outputs=[segmentation])

    losses = {'seg': 'binary_crossentropy'
            }

    metrics = {'seg': ['acc']
                }
    
    
    model.compile(optimizer="adam", loss = losses, metrics=metrics)
    
    
    return model

In [29]:
import datetime

class MyCustomCallback(tf.keras.callbacks.Callback): #acquiring class Callback
    def on_train_begin(self, epoch, logs={}):

        res_dir = "intermediate_results_black_background"

        try:
            os.makedirs(res_dir)
        except:
            print(f"{res_dir} directory already exist")

        print('Training: epoch {} begins at {}'.format(epoch, datetime.datetime.now().time()))

    def on_epoch_end(self, epoch, logs=None):
        res_dir = "intermediate_results_black_background/"
        print('Training: epoch {} ends at {}'.format(epoch, datetime.datetime.now().time()))
        
        for x_test, y_test in keras_generator_train_val_test(batch_size, choice="test"):
            break
        p = np.reshape(x_test[0], (1, 512, 512, 3))
        prediction = self.model.predict(p)

        x_img = f"{res_dir}{epoch}_X_input.jpg"
        y_img = f"{res_dir}{epoch}_Y_truth.jpg"
        predicted_img = f"{res_dir}{epoch}_Y_predicted.jpg"

        cv2.imwrite(x_img, x_test[0] * 255.)
        cv2.imwrite(y_img, y_test['seg'][0] * 255.)
        cv2.imwrite(predicted_img, prediction[0] * 255.)

In [ ]:
from keras.callbacks import Callback 

class NeptuneLogger(Callback):
    def on_batch_end(self, batch, logs=None):  
        for metric_name, metric_value in logs.items():
            neptune.log_metric(name=metric_name, metric_value)

    def on_epoch_end(self, epoch, logs=None): 
        for metric_name, metric_value in logs.items():
            neptune.log_metric(name=metric_name, metric_value)

In [ ]:
# pip install neptune-client neptune-tensorboard

# the same as above
import neptune
from neptunecontrib.monitoring.keras import NeptuneMonitor
 
neptune.init('shared/keras-integration', 'ANONYMOUS')
neptune.create_experiment('keras-training')

model.fit(x_train, y_train, 
          validation_split=0.2, 
          epochs=10, 
          callbacks=[NeptuneMonitor()])

In [11]:
model = get_model()

In [12]:
model.summary()


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 416, 608, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 416, 608, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 416, 608, 32) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 416, 608, 32) 9248        dropout[0][0]                    
_______________________________________________________________________________________

In [ ]:
'''
don't run
'''

### to save best models accross iterations
model_name = "models/"+"Unet_black_background.h5"  #file where to save the best model

modelcheckpoint = ModelCheckpoint(model_name,
                                  monitor='val_loss',
                                  mode='auto',
                                  verbose=1,
                                  save_best_only=True)
###

lr_callback = ReduceLROnPlateau(min_lr=0.000001)

callback_list = [modelcheckpoint, lr_callback, MyCustomCallback()] ## why 3 callbacks??

history = model.fit_generator(
    keras_generator_train_val_test(batch_size, choice="train"),
    validation_data = keras_generator_train_val_test(batch_size, choice="val"),
    validation_steps = 100,
    steps_per_epoch=100,
    epochs=epochs,
    verbose=1, 
    shuffle=True,
    callbacks = callback_list,
)

In [13]:
history = model.fit(x_train, y_train, epochs = epochs, verbose = 1, batch_size = batch_size)

Epoch 1/5
37/37 [==============================] - 663s 18s/step - loss: 0.1134 - acc: 0.2908
Epoch 2/5
37/37 [==============================] - 718s 19s/step - loss: 0.0675 - acc: 0.3199
Epoch 3/5
37/37 [==============================] - 749s 20s/step - loss: 0.0646 - acc: 0.3206
Epoch 4/5
37/37 [==============================] - 704s 19s/step - loss: 0.0638 - acc: 0.3297
Epoch 5/5
37/37 [==============================] - 779s 21s/step - loss: 0.0634 - acc: 0.3328


In [ ]:
## where are the generated images???

## sth to save the weights as they go on??

In [ ]:
def create_mask(pred_mask):
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]

In [ ]:
def show_predictions(dataset=None, num=1):
    if dataset:
        for image, mask in dataset.take(num):
            pred_mask = model.predict(image)
            display([image[0], mask[0], create_mask(pred_mask)])
    else:
        display([sample_image, sample_mask,
             create_mask(model.predict(sample_image[tf.newaxis, ...]))])

In [ ]:
class DisplayCallback(tf.keras.callbacks.Callback): #to see the mask as it goes on
    def on_epoch_end(self, epoch, logs=None):
        clear_output(wait=True)
        show_predictions()
        print ('\nSample Prediction after epoch {}\n'.format(epoch+1))